In [1]:
!nvidia-smi

Sun Feb 25 09:39:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
%%capture
%pip install datasets
%pip install --upgrade pyarrow

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import drive

drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


In [4]:
train_data_path = '/content/drive/MyDrive/kaggle competitions/pii_detection_nlp/data/pii-detection-removal-from-educational-data/train.json'
test_data_path = '/content/drive/MyDrive/kaggle competitions/pii_detection_nlp/data/pii-detection-removal-from-educational-data/test.json'


In [5]:
from tqdm import tqdm
from datasets import load_dataset

In [6]:
# chack datasets was installed correctly:
# output: {'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?', 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}
# note : the output will vary depending on the time the code is run and the updates in the dataset 'squad'
!python3 -c "from datasets import load_dataset; print(load_dataset('squad', split='train')[0])"

Generating train split: 100% 87599/87599 [00:00<00:00, 430000.38 examples/s]
Generating validation split: 100% 10570/10570 [00:00<00:00, 369910.67 examples/s]
{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'question': 'To whom did the Virgin Mary allegedly appear 

In [7]:
train_data = load_dataset('json', data_files=train_data_path, split='train')
train_data

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels'],
    num_rows: 6807
})

In [8]:
train_data.features

{'document': Value(dtype='int64', id=None),
 'full_text': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'trailing_whitespace': Sequence(feature=Value(dtype='bool', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [9]:
example = train_data[0]
pd.DataFrame([example["tokens"][:20], example["labels"][:20]],  # only visualise the 20 first tokens
['Tokens', 'Labels'])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Tokens,Design,Thinking,for,innovation,reflexion,-,Avril,2021,-,Nathalie,Sylla,\n\n,Challenge,&,selection,\n\n,The,tool,I,use
Labels,O,O,O,O,O,O,O,O,O,B-NAME_STUDENT,I-NAME_STUDENT,O,O,O,O,O,O,O,O,O


In [10]:
import time

start_time = time.time()
text = train_data[0]["tokens"]
end_time = time.time()
print(f"Elapsed time: {end_time - start_time:.4f} seconds")

start_time = time.time()
text = train_data["tokens"][0]
end_time = time.time()
print(f"Elapsed time: {end_time - start_time:.4f} seconds")

Elapsed time: 0.0033 seconds
Elapsed time: 4.2486 seconds


In [11]:
# remove irrelevant features
train_data = train_data.remove_columns(["full_text", "trailing_whitespace", "document"])
train_data

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 6807
})

In [43]:
len(train_data[0]['tokens']), len(train_data[0]['labels'])

(753, 753)

In [44]:
tags = train_data.features['labels'].feature
tags

ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-URL_PERSONAL', 'B-ID_NUM', 'B-EMAIL', 'I-STREET_ADDRESS', 'I-PHONE_NUM', 'B-USERNAME', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'I-URL_PERSONAL', 'I-ID_NUM'], id=None)

In [45]:
import datasets

train_data = train_data.cast_column("labels", datasets.Sequence(datasets.ClassLabel(names=['O',
 'B-NAME_STUDENT',
 'I-NAME_STUDENT',
 'B-URL_PERSONAL',
 'B-ID_NUM',
 'B-EMAIL',
 'I-STREET_ADDRESS',
 'I-PHONE_NUM',
 'B-USERNAME',
 'B-PHONE_NUM',
 'B-STREET_ADDRESS',
 'I-URL_PERSONAL',
 'I-ID_NUM'])))
train_data.features

Casting the dataset:   0%|          | 0/6807 [00:00<?, ? examples/s]

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-URL_PERSONAL', 'B-ID_NUM', 'B-EMAIL', 'I-STREET_ADDRESS', 'I-PHONE_NUM', 'B-USERNAME', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'I-URL_PERSONAL', 'I-ID_NUM'], id=None), length=-1, id=None),
 'name_labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT'], id=None), length=-1, id=None)}

In [46]:
example = train_data[0]
pd.DataFrame([example["tokens"][:20], example["labels"][:20]],  # only visualise the 20 first tokens
['Tokens', 'Labels'])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Tokens,Design,Thinking,for,innovation,reflexion,-,Avril,2021,-,Nathalie,Sylla,\n\n,Challenge,&,selection,\n\n,The,tool,I,use
Labels,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0


In [48]:
tags = train_data.features['labels'].feature
tags

ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-URL_PERSONAL', 'B-ID_NUM', 'B-EMAIL', 'I-STREET_ADDRESS', 'I-PHONE_NUM', 'B-USERNAME', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'I-URL_PERSONAL', 'I-ID_NUM'], id=None)

In [49]:
# We only need the names for longformer finetunining -> rename all other labels as Other?
# all labels that are not names become 'O'
# we only need the tags 0 , 1, 2

def replace_non_names(example):
  example['labels']
  names = [1, 2]
  name_labels_lists = []
  for label in example['labels']:
    if label in names:
        name_labels_lists.append(label)
    else:
        name_labels_lists.append(0)
  example['name_labels'] = name_labels_lists
  return example

In [50]:
train_data = train_data.map(replace_non_names)

Map:   0%|          | 0/6807 [00:00<?, ? examples/s]

In [18]:
train_data

Dataset({
    features: ['tokens', 'labels', 'name_labels'],
    num_rows: 6807
})

In [53]:
len(train_data[0]['tokens']), len(train_data[0]['labels']), len(train_data[0]['name_labels'])

(753, 753, 753)

In [54]:
train_data.features['name_labels'].feature

ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT'], id=None)

In [55]:
train_data = train_data.cast_column("name_labels", datasets.Sequence(datasets.ClassLabel(names=['O',
 'B-NAME_STUDENT',
 'I-NAME_STUDENT'])))
train_data.features

Casting the dataset:   0%|          | 0/6807 [00:00<?, ? examples/s]

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-URL_PERSONAL', 'B-ID_NUM', 'B-EMAIL', 'I-STREET_ADDRESS', 'I-PHONE_NUM', 'B-USERNAME', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'I-URL_PERSONAL', 'I-ID_NUM'], id=None), length=-1, id=None),
 'name_labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT'], id=None), length=-1, id=None)}

In [57]:
tags = train_data.features['name_labels'].feature
tags

ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT'], id=None)

In [58]:
example = train_data[0]
pd.DataFrame([example["tokens"][:20], example["name_labels"][:20]],  # only visualise the 20 first tokens
['Tokens', 'Labels'])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Tokens,Design,Thinking,for,innovation,reflexion,-,Avril,2021,-,Nathalie,Sylla,\n\n,Challenge,&,selection,\n\n,The,tool,I,use
Labels,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0


In [59]:
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}

In [60]:
tags.num_classes

3

In [61]:
dataset = train_data.train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'name_labels'],
        num_rows: 5445
    })
    test: Dataset({
        features: ['tokens', 'labels', 'name_labels'],
        num_rows: 1362
    })
})

In [62]:
dataset['train'].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'B-URL_PERSONAL', 'B-ID_NUM', 'B-EMAIL', 'I-STREET_ADDRESS', 'I-PHONE_NUM', 'B-USERNAME', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'I-URL_PERSONAL', 'I-ID_NUM'], id=None), length=-1, id=None),
 'name_labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT'], id=None), length=-1, id=None)}

In [63]:
len(dataset['train'][0]['tokens']), len(dataset['train'][0]['labels']), len(dataset['train'][0]['name_labels'])

(709, 709, 709)

In [64]:
def remove_col(dataset, col):
  for split in dataset:
    dataset[split] = dataset[split].remove_columns([col])
  return dataset

remove_col(dataset, 'labels')


DatasetDict({
    train: Dataset({
        features: ['tokens', 'name_labels'],
        num_rows: 5445
    })
    test: Dataset({
        features: ['tokens', 'name_labels'],
        num_rows: 1362
    })
})

In [65]:
dataset['train'].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'name_labels': Sequence(feature=ClassLabel(names=['O', 'B-NAME_STUDENT', 'I-NAME_STUDENT'], id=None), length=-1, id=None)}

In [66]:
# https://huggingface.co/hewonty/longformer-ner-finetuned-pii - fine tuned already
# https://huggingface.co/allenai/longformer-base-4096 - original model
from transformers import AutoTokenizer

lf_model = 'allenai/longformer-base-4096'
lf_tokenizer = AutoTokenizer.from_pretrained(lf_model, add_prefix_space=True)

In [67]:
len(dataset['train'][0]['tokens']), len(dataset['train'][0]['name_labels'])

(709, 709)

In [76]:
text = dataset['train'][0]['tokens']
tokenized_input = lf_tokenizer(text, is_split_into_words=True)
lf_tokens = lf_tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
pd.DataFrame({
    'tokens': dataset['train'][0]['tokens'][:10],
    'lf_tokens': lf_tokens[:10],
    'name_labels': dataset['train'][0]['name_labels'][:10]
}).T

,0,1,2,3,4,5,6,7,8,9
tokens,Design,Thinking,for,Innovation,,UdelaR,,Edith,Orjuela,\n\n
lf_tokens,<s>,ĠDesign,ĠThinking,Ġfor,ĠInnovation,Ġ,ĠUd,ela,R,Ġ
name_labels,0,0,0,0,0,0,0,1,2,0


In [69]:
len(dataset['train'][0]['tokens']), len(dataset['train'][0]['name_labels']), len(lf_tokens)

(709, 709, 799)

In [71]:
len(dataset['train'][0]['name_labels']), len(text), len(lf_tokens)

(709, 709, 799)

In [92]:
# the tokenized 'tokens' result in a longer list - the logformer tokenizer splits into subwords
# we need to align the tokens when we tokenize

def tokenize_and_align_labels(examples):
    tokenized_inputs = lf_tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"name_labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["name_tags"] = labels
    return tokenized_inputs


In [93]:
for split in dataset:
  dataset[split] = dataset[split].map(tokenize_and_align_labels, batched=True)
dataset

Map:   0%|          | 0/5445 [00:00<?, ? examples/s]

Map:   0%|          | 0/1362 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'name_labels', 'input_ids', 'attention_mask', 'name_tags'],
        num_rows: 5445
    })
    test: Dataset({
        features: ['tokens', 'name_labels', 'input_ids', 'attention_mask', 'name_tags'],
        num_rows: 1362
    })
})

In [97]:
pd.DataFrame({
    'tokens': dataset['train'][0]['tokens'][:15],
    'name_labels': dataset['train'][0]['name_labels'][:15],
    'input_ids/tokens': lf_tokenizer.convert_ids_to_tokens(dataset['train'][0]['input_ids'][:15]),
    'name_tags': dataset['train'][0]['name_tags'][:15],
}).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
tokens,Design,Thinking,for,Innovation,,UdelaR,,Edith,Orjuela,\n\n,Challenge,,Since,I,am
name_labels,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0
input_ids/tokens,<s>,ĠDesign,ĠThinking,Ġfor,ĠInnovation,Ġ,ĠUd,ela,R,Ġ,ĠEd,ith,ĠOr,j,uel
name_tags,-100,0,0,0,0,0,0,-100,-100,0,1,-100,2,-100,-100


In [98]:
len(dataset['train'][0]['tokens']), len(dataset['train'][0]['name_labels'])

(709, 709)

In [99]:
len(dataset['train'][0]['input_ids']), len(dataset['train'][0]['name_tags'])

(799, 799)

In [103]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'name_labels', 'input_ids', 'attention_mask', 'name_tags'],
        num_rows: 5445
    })
    test: Dataset({
        features: ['tokens', 'name_labels', 'input_ids', 'attention_mask', 'name_tags'],
        num_rows: 1362
    })
})

In [104]:
remove_col(dataset, 'tokens')
remove_col(dataset, 'name_labels')
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'name_tags'],
        num_rows: 5445
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'name_tags'],
        num_rows: 1362
    })
})

## Create the model for token classification

In [ ]:
import torch.nn as nn
from transformers import LongformerConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel

In [102]:
from transformers import AutoConfig

lf_config = AutoConfig.from_pretrained(lf_model,
                                     num_labels=tags.num_classes,
                                     id2label=index2tag, label2id=tag2index)
